In [1]:
from nlptoolkits.FirmLevelRiskKits import DictionaryT, PreprocessT
from nlptoolkits.SmallKits import IOHandlerT

We make the basic classes for further use

In [2]:
internation_tax_dictionary = IOHandlerT.file_to_list(
    './input_data/annotatedsent_Principles_of_International_Taxation.txt',
    charset_error_encoding='utf-8'
)[:1000]

tweets_texture = IOHandlerT.file_to_list(
    './input_data/tweets_origin.txt',
    charset_error_encoding='utf-8'
)[:1000]

# make a class to clean the data(dict)
corenlp_preprocess_cls = PreprocessT.NgramDataPreprocessor()

# make a class to build the dictionary
corenlp_ngram_dict_cls = DictionaryT.NgramDictionaryBuilder()

In [3]:
internation_tax_dictionary[:3]

['the[POS:DT] principal[POS:JJ] aim[POS:NN] of[POS:IN] this[POS:DT] book[POS:NN] be[POS:VBZ] to[POS:TO] [NER:DATE]present[POS:VB] the[POS:DT] topic[POS:NN] in[POS:IN] a[POS:DT] accessible[POS:JJ] manner[POS:NN] to[POS:IN] that[POS:DT] who[POS:WP] be[POS:VBP] fairly[POS:RB] new[POS:JJ] to[POS:IN] international[POS:JJ] taxation[POS:NN] or[POS:CC] ,[POS:,] indeed[POS:RB] ,[POS:,] be[POS:VBP] new[POS:JJ] to[POS:IN] taxation[POS:NN] in[POS:IN] any[POS:DT] guise[POS:NN] .[POS:.]',
 'it[POS:PRP] will[POS:MD] be[POS:VB] useful[POS:JJ] for[POS:IN] university[POS:NN][SEP]student[POS:NNS] ,[POS:,] for[POS:IN] that[POS:DT] prepare[POS:VBG] for[POS:IN] professional[POS:JJ] qualification[POS:NNS] ,[POS:,] and[POS:CC] for[POS:IN] practitioner[POS:NNS] generally[POS:RB] .[POS:.]',
 '[NER:PERSON]Angharad[POS:NNP][SEP]Miller[POS:NNP] and[POS:CC] I[POS:PRP] [NER:ORDINAL]first[POS:RB] embark[POS:VBD] on[POS:IN] this[POS:DT] project[POS:NN] in[POS:IN] [NER:DATE]2006[POS:CD] ,[POS:,] at[POS:IN] she[POS:PRP$

In [4]:
tweets_texture[:3]

['"This year, throw kindness around like confetti! Happy New Year!  There is only one Kennan Fritz and we\'re sad to see him go! Kennan, you\'ve been part of the Independence Bank family for almost two decades and in many of our markets. You will be missed by all of us as we celebrate your retirement and wish you well!  REMINDER: Be aware of calls asking you to do anything you wouldn<U+2019>t normally do with your online account/debit card info. Don\'t disclose any personal information unless you\'re calling the bank and initiating. The bank will never call you and initiate the request for information.  Mackenzie Metcalf is prepared and qualified to continue serving clients as a Financial Advisor. Mackenzie is thrilled to advance her career by educating people on products and services offered by what she considers to be the best bank in Kentucky.   Don<U+2019>t forget! If you can<U+2019>t get out or prefer to stay at home we offer many convenient ways to bank from the comfort of your o

Part to make Ngram dictionary

In [5]:
international_tax_tfidf = corenlp_ngram_dict_cls.n_gramizer_dictionary_builder(internation_tax_dictionary, scorer = 'tfidf', 
                                                                               remove_ngram_postags_combinations=None
                                                                               )

100%|██████████| 12133/12133 [00:06<00:00, 1864.42it/s]


In [6]:
international_tax_tfidf

{('resident', 'as'): 1,
 ('available', 'accounting'): 1,
 ('angharad', 'miller'): 1,
 ('kong', 'and'): 1,
 ('therefore', 'in'): 2,
 ('name', 'but'): 1,
 ('generally', 'accept'): 1,
 ('compute', 'tax'): 1,
 ('by', 'expectation'): 1,
 ('consult', 'in'): 1,
 ('of', 'paper'): 1,
 ('study', 'by'): 2,
 ('report', 'provide'): 1,
 ('other', 'greenhouse'): 1,
 ('departure', 'and'): 1,
 ('include', 'hungary'): 1,
 ('be', 'only'): 5,
 ('tax', 'they'): 3,
 ('understand', 'how'): 1,
 ('the', 'component'): 1,
 ('equally', 'wealth'): 1,
 ('behave', 'compliance'): 1,
 ('in', 'we'): 1,
 ('of', 'knowledge'): 1,
 ('the', 'poor'): 1,
 ('country', 'company'): 1,
 ('any', 'case'): 1,
 ('employee', 'improbable'): 1,
 ('more', 'robust'): 1,
 ('relate', 'in'): 1,
 ('profit', 'before'): 1,
 ('con', 'thus'): 1,
 ('but', 'student'): 1,
 ('country', 'it'): 5,
 ('haven', 'country'): 1,
 ('expenditure', 'they'): 1,
 ('will', 'pay'): 1,
 ('factor', 'in'): 1,
 ('seebraithwaite', 'although'): 1,
 ('the', 'incentive'): 

You can remove words by remove phrases which are in another dictionary, but we do not do here now.

We now clean the tweets data quickly by using the nltk quick cleaner

In [16]:
PreprocessT.naive_nltk_annotator(
    tweets_texture
)[0]

"`` This year , throw kindness around like confetti ! Happy New Year ! There is only one Kennan Fritz and we 're sad to see him go ! Kennan , you 've been part of the Independence Bank family for almost two decades and in many of our markets . You will be missed by all of us as we celebrate your retirement and wish you well ! REMINDER : Be aware of calls asking you to do anything you wouldn < U+2019 > t normally do with your online account/debit card info . Do n't disclose any personal information unless you 're calling the bank and initiating . The bank will never call you and initiate the request for information . Mackenzie Metcalf is prepared and qualified to continue serving clients as a Financial Advisor . Mackenzie is thrilled to advance her career by educating people on products and services offered by what she considers to be the best bank in Kentucky . Don < U+2019 > t forget ! If you can < U+2019 > t get out or prefer to stay at home we offer many convenient ways to bank from

In [17]:
corenlp_preprocess_cls.clean_annotated_texture_list(PreprocessT.naive_nltk_annotator(
    tweets_texture
), processes=5)[0]

Cleaning the annotated data...


100%|██████████| 1000/1000 [00:10<00:00, 94.98it/s]


'this year throw kindness around like confetti happy new year there is only one kennan fritz and we sad to see him go kennan you been part of the independence bank family for almost two decades and in many of our markets you will be missed by all of us as we celebrate your retirement and wish you well reminder be aware of calls asking you to do anything you wouldn normally do with your online card info do disclose any personal information unless you calling the bank and initiating the bank will never call you and initiate the request for information mackenzie metcalf is prepared and qualified to continue serving clients as financial advisor mackenzie is thrilled to advance her career by educating people on products and services offered by what she considers to be the best bank in kentucky don forget if you can get out or prefer to stay at home we offer many convenient ways to bank from the comfort of your own home digitalbanking telephonebanking website stay safe everyone the independe